<a href="https://colab.research.google.com/github/EstebanmAcero/Neural_nets/blob/main/Ejercicio_2_Red_Neuronal_desde_cero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Luis Esteban Molina Acero 201910203**
#**Manuel Velez PhD.**


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_circles

# Crear el dataset
n = 500
p = 2
X, Y = make_circles(n_samples=n, factor=0.5, noise=0.03)

# Graficar los datos
plt.scatter(X[Y == 0, 0], X[Y == 0, 1], c="skyblue")
plt.scatter(X[Y == 1, 0], X[Y == 1, 1], c="salmon")
plt.axis("equal")
plt.show()

# Definición de la clase para una capa de la red neuronal
class neural_layer():
    def __init__(self, n_conn, n_neur, act_f):
        self.act_f = act_f
        self.b = np.random.rand(1, n_neur) * 2 - 1  # Inicialización aleatoria de los sesgos
        self.W = np.random.rand(n_conn, n_neur) * 2 - 1  # Inicialización aleatoria de los pesos

# Definición de funciones de activación (Sigmoid y su derivada)
sigm = (lambda x: 1 / (1 + np.exp(-x)), lambda x: x * (1 - x))

# Crear una capa de la red neuronal
_l0 = neural_layer(p, 4, sigm)
_l1 = neural_layer(4, 8, sigm)

# Función para crear una red neuronal con la topología especificada
def create_nn(topology, act_f):
    nn = []
    for l, layer in enumerate(topology[:-1]):
        nn.append(neural_layer(topology[l], topology[l + 1], act_f))
    return nn

# Definir la topología de la red y crear la red
topology = [p, 4, 8, 16, 32, 8, 4, 1]
neural_net = create_nn(topology, sigm)

# Función de costo (Error cuadrático medio)
l2_cost = (lambda Yp, Yr: np.mean((Yp - Yr) ** 2), lambda Yp, Yr: (Yp - Yr))

def train(neural_net, X, Y, l2_cost, lr=0.5):
    out = [(None, X)]

    # Forward pass (propagación hacia adelante)
    for l, layer in enumerate(neural_net):
        z = out[-1][1] @ neural_net[l].W + neural_net[l].b  # Combinación lineal de entradas y pesos
        a = neural_net[l].act_f[0](z)  # Aplicación de la función de activación
        out.append((z, a))  # Guardar las salidas intermedias

    print("Costo inicial:", l2_cost[0](out[-1][1], Y))

    if train:
        deltas = []

        # Backward pass (propagación hacia atrás)
        for l in reversed(range(0, len(neural_net))):
            z = out[l + 1][0]
            a = out[l + 1][1]

            if l == len(neural_net) - 1:
                deltas.insert(0, l2_cost[1](a, Y) * neural_net[l].act_f[1](a))  # Calcular el error y la derivada
            else:
                deltas.insert(0, deltas[0] @ neural_net[l + 1].W.T * neural_net[l].act_f[1](a))

            _w = neural_net[1].W

            # Descenso del gradiente
            neural_net[l].b = neural_net[l].b - np.mean(deltas[0], axis=0, keepdims=True) * lr
            neural_net[l].W = neural_net[l].W - out[l][1].T @ deltas[0] * lr

    print("Costo final:", l2_cost[0](out[-1][1], Y))
    return out[-1][1]

# Entrenar la red neuronal
train(neural_net, X, Y, l2_cost, 0.5)


In [ ]:
import time
from IPython.display import clear_output

neural_n = create_nn(topology, sigm)

loss = []

for i in range(1):
    # Entrenamos a la red neuronal
    pY = train(neural_n, X, Y, l2_cost)

    if i % 25 == 0:
        loss.append(l2_cost[0](pY, Y))

        res = 50
        _x0 = np.linspace(-1.5, 1.5, res)
        _x1 = np.linspace(-1.5, 1.5, res)
        Y_ = np.zeros((res, res))

        for i0, x0 in enumerate(_x0):
            for i1, x1 in enumerate(_x1):
                Y_[i0, i1] = train(neural_n, np.array([[x0, x1]]), Y, l2_cost)[0][0]

        plt.pcolormesh(_x0, _x1, Y_, cmap="coolwarm")
        plt.axis("equal")
        plt.scatter(X[Y == 0, 0], X[Y == 0, 1], c="skyblue")
        plt.scatter(X[Y == 1, 0], X[Y == 1, 1], c="salmon")

        clear_output(wait=True)
        plt.plot(range(len(loss)), loss)
        plt.show()
        time.sleep(0.5)
